#### Search

In [ ]:
%pip install -q azure-search-documents python-dotenv

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient

import os
from dotenv import load_dotenv

load_dotenv("../.env")

# Azure Search service details
service_name = os.getenv("SERVICE_NAME")
admin_key = os.getenv("SEARCH_ADMIN_KEY")
index_name = os.getenv("SEARCH_INDEX_NAME")

# Initialize the search index client
endpoint = f"https://{service_name}.search.windows.net/"
credential = AzureKeyCredential(admin_key)
index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)

print(f"endpoint: {endpoint}")
print(f"index_name: {index_name}")

In [3]:
from openai import OpenAI

openai_client = OpenAI()

def embed_query(query: str) -> list:
    response = openai_client.embeddings.create(
        input=query, model="text-embedding-3-small"
    )
    return response.data[0].embedding

In [5]:
from azure.search.documents.models import VectorizedQuery
from azure.search.documents.models import QueryType, QueryCaptionType, QueryAnswerType

def full_text_search(query, top=5):
    """Perform a full-text search."""
    results = search_client.search(search_text=query, top=top)
    return list(results)

def vector_search(query):
    embedded_query = embed_query(query)
    vector_query = VectorizedQuery(vector=embedded_query, k_nearest_neighbors=3, fields="embedding")
  
    results = search_client.search(  
        search_text=None,  
        vector_queries= [vector_query],
        select=["hotel_name", "review_text", "review_title", "city"],
    )  
    return list(results)

def hybrid_search(query, top=5):
    """Perform a hybrid search (combination of full-text and vector search)."""
    embedded_query = embed_query(query)
    vector_query = VectorizedQuery(vector=embedded_query, k_nearest_neighbors=3, fields="embedding")
    results = search_client.search(
        search_text=query,
        vector_queries=[vector_query],
        select=["id", "review_text", "review_title", "hotel_name", "city"],
        top=top,
    )
    return list(results)

def semantic_search(query, top=5):
    """Perform a semantic search using the vector search method."""
    embedded_query = embed_query(query)
    vector_query = VectorizedQuery(
        vector=embedded_query, k_nearest_neighbors=3, fields="embedding"
    )
    results = search_client.search(
        search_text=query,
        vector_queries=[vector_query],
        select=[
            "id",
            "review_text",
            "review_title",
            "hotel_name",
            "city",
            "hotel_state",
        ],
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name="ps-hotels-semantic-config",
        query_caption=QueryCaptionType.EXTRACTIVE,
        query_answer=QueryAnswerType.EXTRACTIVE,
        top=top,
    )
    return results

def print_semantic_results(results):
    for result in results:
        print(f"Hotel Name: {result['hotel_name']}")
        print(f"Reranker Score: {result['@search.reranker_score']}")
        print(f"Review Title: {result['review_title']}")
        print(f"Review: {result['review_text']}")

        captions = result["@search.captions"]
        if captions:
            caption = captions[0]
            if caption.highlights:
                print(f"Caption: {caption.highlights}")
            else:
                print(f"Caption: {caption.text}")
        print("\n")

def print_results(results):
    for result in results:
        print(f"Hotel Name: {result['hotel_name']}")
        print(f"City: {result['city']}")
        print(f"Score: {result['@search.score']}")
        print(f"Review Title: {result['review_title']}")
        print(f"Review: {result['review_text']}")
        print("\n")

In [ ]:
query = "walkable"
results = semantic_search(query)

print_semantic_results(results)


#### Summarize Using OpenAI

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

SYSTEM_MESSAGE = "You are a helpful assistant."

USER_QUERY = query
SEARCH_RESULTS = results

USER_MESSAGE = f"""
  You are provided a user query, and the search results based on user query. You task is to summarize the results and put the best order for the results. 

USER_QUERY
```
{USER_QUERY}
```

SEARCH_RESULTS
```
{SEARCH_RESULTS}
```

Return a valid JSON with the following information

RESULT_SCHEMA
```
{{
  'summary': '',
  'results': []
}}
```
"""


response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
    response_format={"type": "json_object"},
)

response = json.loads(response.choices[0].message.content) 
print(json.dumps(response, indent=2))